In [ ]:
#Name: Zaynab Batool Reza
#Email: i180419@nu.edu.pk

## Importing Required Libraries

In [54]:
!pip install -U scikit-learn

     |████████████████████████████████| 22.3MB 1.5MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [65]:
import spacy as sp
import os
import copy
import math
import sklearn.metrics as sk
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/NLP/Assignment5/Train

/content/gdrive/MyDrive/NLP/Assignment5/Train


## Utility Functions to Implement Algorithm

### Fetch Training Data
This function first declares a dictionary calles **"classes"** which will store all the texts against what class they belong to as their key.

Firstly, it gets the names of classes by getting list of filenames in the Train folder which are Real and Fake in this scenario.

Then in it adds that class to the keys of the **classes** dictionary and initializes the items against it as an empty list.

Following that, each folder for each class is traversed and all the texts files are read, normalized by removing any punctuation, extra spaces or letters and then appended to the list against the class they belong to.

At the end the dictionary called **classes** is returned which has list of all texts fetched for training as items against the class they belong to as keys.


In [6]:
#get all real and fake documents as lists
def getData():
  %cd /content/gdrive/MyDrive/NLP/Assignment5/Train
  punc='، ‘ ، ! ’ % ٪ ؟ : \ “. ۔  ) ( \ ” \" \' '
  punclist= [' ؔ','َ','ِ','ُ',]
  for p in punclist:
    if p not in punc:
      punc+=" "+p
  punc=punc.split()
  alphabets='ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
  classes={}
  for classNames in os.listdir(os.getcwd()):
    print("Getting documents for "+classNames)
    #path='/content/gdrive/MyDrive/NLP/Assignment5/Train/'+classNames
    %cd {classNames}
    classes[classNames]=[]
    for i, docName in enumerate(os.listdir(os.getcwd())):
      # print(i)
      #print(docName)
      file=open(docName, 'r') 
      text=file.read()
      # clean text of extra spaces, end of line char and punc, check code from poetry
      text=text.replace('\n', " ")   
      for char in text:
        if char in punc:
          text = text.replace(char, " ")
        if char in alphabets:
          text=text.replace(char, " ")
      text= " ".join(text.split())   
      # cleaning ends here
      classes[classNames].append(text)
    %cd /content/gdrive/MyDrive/NLP/Assignment5/Train
  return classes
    #with open(os.path.join(os.getcwd(), filename), 'r') as f:

# classes=getData()
# print(len(classes['Real']))

### Calculate Prior of Each Class
This function calculates what the prior of each class is and stores it in a dictionary called classPrior against the respective class as the key.

It takes the dictionary that contains all texts against their class as a parameter and firstly counts total number of texts

Then it calculates the classPrior for each class by dividing the number of texts in that class with the total number of texts and stores it in the dictionary with the class Name as its key.

The classPrior dictionary is then returned

In [7]:
def getClassPrior(classes):
  classPrior={}
  totalTexts=0
  for className in classes:
    totalTexts+=len(classes[className])
  for className in classes:
    classPrior[className]=(len(classes[className]))/float(totalTexts)
  return classPrior


### Concatenate Texts of Each Class
This function returns a dictionary in which the keys are the class names and the items stored against them are all the texts in that class concatenated together.

It takes as input parameter the dictionary that has all texts stored against which class they belong to.

Next for each class, it concatenates all the texts and then assigns the resulting string to the new **concatenatedClasses** dictionary against the class name as key

In [8]:
def contatenateTexts(classesp):
  concatenatedClasses={}
  for className in classesp:
    concatenatedClasses[className]=""
    for text in classesp[className]:
      concatenatedClasses[className]+=" "+text
  return concatenatedClasses


### Calculating Conditional Probabilities of Words
This function calculates the conditional probability of each word for each class and returns a nested dictionary which has the classes as keys. Against every class there is another dictionary which has the words as keys and their conditional probability for that class as the items.

Firstly, after taking the concatenatedClasses as input parameter which is a dictionary having all texts concatenated together against the class they belong to, it extracts the vocabulary.

Next, for every class in the concatenatedClasses dictionary, it initializes the inner dictionary that will contain the words as empty.

Then for every word in the vocabulary, it counts number of times it appears in that class and after applying laplace smoothing and dividing it with the total number if words in that class, the conditional probability of that words in that class is saved in the nested dictionary named **wordProbabilitiesByClass**.

This dictionary is then returned at the end.

An example of how the probabilities are stored and accessed:

wordProbabilitiesByClass={'Real':{'w1':1, 'w2':3, 'w3':2}, 'Fake':{'w1':2,'w2':2, 'w3': 1}}

Then to get probability of w1 in Real, prob=wordProbabilitiesByClass['Real']['w1']

In [22]:
def getWordInClassProb(concatenatedClasses):
  vocabulary=[]
  wordProbabilitiesByClass={}
  c=0
  for className in concatenatedClasses:
    print("In class ", className)
    document=concatenatedClasses[className]   
    udoc=document.split()      
    for w in udoc:
      if w not in vocabulary:
        vocabulary.append(w)  
  
  #outer dictionary has classes as keys, in each value there is another dictionary where words are keys and their probability for that class is their value
  for className in concatenatedClasses:
    print("In class ", className)
    wordProbabilitiesByClass[className]={}
    print("Vocabulary size: ", len(vocabulary))
    for i in range(len(vocabulary)):
      w=vocabulary[i]
      occurenceInClass=0
      document=concatenatedClasses[className] 
      udoc=document.split()      
      occurenceInClass=udoc.count(w)
      wordProbabilitiesByClass[className][w]=(occurenceInClass+1)/float(len(concatenatedClasses[className].split())+len(vocabulary))

  return wordProbabilitiesByClass, vocabulary
  


### Fetch Testing Data
This function first declares a dictionary calles **"testClasses"** which will store all the test texts against what class they belong to as their key.

Firstly, it gets the names of classes by getting list of filenames in the Test folder which are Real and Fake in this scenario.

Then in it adds that class to the keys of the **testClasses** dictionary and initializes the items against it as an empty list.

Following that, each folder for each class is traversed and all the texts files are read, normalized by removing any punctuation, extra spaces or letters and then appended to the list against the class they belong to.

At the end the dictionary called **testClasses** is returned which has list of all texts fetched for testing as items against the class they belong to as keys.

In [11]:
def getTestData():
  %cd /content/gdrive/MyDrive/NLP/Assignment5/Test
  punc='، ‘ ، ! ’ % ٪ ؟ : \ “. ۔  ) ( \ ” \" \' '
  punclist= [' ؔ','َ','ِ','ُ',]
  for p in punclist:
    if p not in punc:
      punc+=" "+p
  punc=punc.split()
  alphabets='ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
  testClasses={}
  for classNames in os.listdir(os.getcwd()):
    print("Getting documents for "+classNames)
    %cd {classNames}
    testClasses[classNames]=[]
    for i, docName in enumerate(os.listdir(os.getcwd())):
      file=open(docName, 'r') 
      text=file.read()
      # clean text of extra spaces, end of line char and punc, check code from poetry
      text=text.replace('\n', " ")   
      for char in text:
        if char in punc:
          text = text.replace(char, " ")
        if char in alphabets:
          text=text.replace(char, " ")
      text= " ".join(text.split())   
      # cleaning ends here
      testClasses[classNames].append(text)
    %cd /content/gdrive/MyDrive/NLP/Assignment5/Test
  return testClasses


## Naive Bayes Training Function

The function uses the functions defined above and after taking as input the dictionary containing all texts, gets the Class Priors, Conditional Probabilities and the Vocabulary for those texts.

In [23]:
def NaiveBaiseTraining(classesp):
  classPrior=getClassPrior(classesp)
  concatenatedClasses=contatenateTexts(classesp)
  wordProbabilitiesByClass, vocabulary=getWordInClassProb(concatenatedClasses)
  return vocabulary,classPrior, wordProbabilitiesByClass


## Naive Bayes Testing Function
This function takes as input parameters, the dictionary containing all the texts for training against their class names, the vocabulary, the dictionary having class priors, the dictionary having conditional probabilities and the text the algorithm is to be tested on.

First all words that also belong to vocabulary are fetched from the text.

Next, a dictionary called scored is intialized with having the class name as keys and the score for that text belonging to that class against it.

Initially the score for each class is the log of the class Prior of that class and then for every word extracted from the text, the log of its conditional probability for occuring in that class is added.

In the end, the class with the highest score is returned as the predicted class for that text

In [40]:
def naiveBayesTest(testClasses, v, prior, prob, t):
  tempwords=t.split()
  words=[]
  for word in tempwords:
    if word in v:
      words.append(word)
  score={}
  for className in testClasses:
    score[className]=math.log(prior[className])
    for w in words:
      score[className]+=math.log(prob[className][w])
  return max(score, key=score.get)

In [12]:
classes=getData() #put in separate cell in future so data is fetched just once

/content/gdrive/MyDrive/NLP/Assignment5/Train
Getting documents for Real
/content/gdrive/MyDrive/NLP/Assignment5/Train/Real
/content/gdrive/MyDrive/NLP/Assignment5/Train
Getting documents for Fake
/content/gdrive/MyDrive/NLP/Assignment5/Train/Fake
/content/gdrive/MyDrive/NLP/Assignment5/Train


In [13]:
testClasses=getTestData()

/content/gdrive/MyDrive/NLP/Assignment5/Test
Getting documents for Real
/content/gdrive/MyDrive/NLP/Assignment5/Test/Real
/content/gdrive/MyDrive/NLP/Assignment5/Test
Getting documents for Fake
/content/gdrive/MyDrive/NLP/Assignment5/Test/Fake
/content/gdrive/MyDrive/NLP/Assignment5/Test


# Training 

## Normal Naive Baise Training

Will take approximately 10-11 minutes to execute

In [24]:
vocabulary, classPrior, wordProbabilitiesByClass=NaiveBaiseTraining(classes)
print(len(wordProbabilitiesByClass['Real']))
print(wordProbabilitiesByClass['Fake']['جس'])
print(wordProbabilitiesByClass['Real']['جس'])

In class  Real
In class  Fake
In class  Real
Vocabulary size:  13750
In class  Fake
Vocabulary size:  13750
13750
0.0034291477559253657
0.002564304593543396


## Stop Words Removed Training

This function removes all stop words from all the texts in the input dictionary and returns the new dictionary

In [32]:
def removeStopWords(classesp):
  %cd /content/gdrive/MyDrive/NLP/Assignment5
  f= open('stopwords-ur.txt', 'r')
  stopWords=f.read()
  stopWords=stopWords.split()
  stopWordsRemovedClasses={}
  for className in classesp:
    textsList=copy.deepcopy(classesp[className])
    newtextsList=[]
    for texts in textsList:
      newtext=""
      allWords=texts.split()
      temp=[]
      for word in allWords:
        if word not in stopWords:
          temp.append(word)
      newtext=' '.join(temp)
      newtextsList.append(newtext)
    stopWordsRemovedClasses[className]=newtextsList

  return stopWordsRemovedClasses


Removing stop words from training and test data sets and storing in new dictionary

In [33]:
stopWordsRemovedClasses=removeStopWords(classes)
stopWordsRemovedTest=removeStopWords(testClasses)

/content/gdrive/MyDrive/NLP/Assignment5
/content/gdrive/MyDrive/NLP/Assignment5


Will take approximately 8-10 minutes to execute

In [100]:
vocabularySW, classPriorSW, wordProbabilitiesByClassSW=NaiveBaiseTraining(stopWordsRemovedClasses)
print(len(wordProbabilitiesByClassSW['Real']))
print(wordProbabilitiesByClassSW['Fake']['جس'])
print(wordProbabilitiesByClassSW['Real']['جس'])

In class  Real
In class  Fake
In class  Real
Vocabulary size:  13588
In class  Fake
Vocabulary size:  13588
13588
0.0042860024203307786
0.0032414910858995136


## PreProcessed Negations Training

This function processes negations by removing words that come after negatory words.

In [86]:
def processNegations(classesp):
  negationClasses={}
  for className in classesp:
    textsList=copy.deepcopy(classesp[className])
    newtextsList=[]
    for texts in textsList:
      newtext=""
      allWords=texts.split()
      temp = []
      for i,word in enumerate(allWords):
        if i>0:
          if allWords[i-1]!='نہیں': # get rid of the word after nahi e.g انہوں نے نہیں بولا will become انہوں نے نہیں
            temp.append(word)
      newtext= ' '.join(temp)
      newtextsList.append(newtext)
    negationClasses[className]=newtextsList

  return negationClasses

Deaking with negations in both training and testing sets and storing in new dictionaries

In [87]:
negationTrainingClasses=processNegations(classes)
negationTestingClasses=processNegations(testClasses)

Will take approximately 10-11 minutes to execute

In [88]:
vocabularyNG, classPriorNG, wordProbabilitiesByClassNG=NaiveBaiseTraining(negationTrainingClasses)
print(len(wordProbabilitiesByClassNG['Real']))
print(wordProbabilitiesByClassNG['Fake']['جس'])
print(wordProbabilitiesByClassNG['Real']['جس'])

In class  Real
In class  Fake
In class  Real
Vocabulary size:  13637
In class  Fake
Vocabulary size:  13637
13637
0.0034564085881587507
0.0025839420273118343


## PreProcessed Negations+Removal of StopWords Together Training

Removing stop words from both training and testing sets where negations have been preprocessed

In [89]:
stopWordsNGRemovedClasses=removeStopWords(negationTrainingClasses)
stopWordsNGRemovedTest=removeStopWords(negationTestingClasses)

/content/gdrive/MyDrive/NLP/Assignment5
/content/gdrive/MyDrive/NLP/Assignment5


In [90]:
vocabularySWNG, classPriorSWNG, wordProbabilitiesByClassSWNG=NaiveBaiseTraining(stopWordsNGRemovedClasses)

In class  Real
In class  Fake
In class  Real
Vocabulary size:  13475
In class  Fake
Vocabulary size:  13475


## Boolean Naive Baise Training (Removing Dupliacate Words)

This function removes all stop words from all the texts in the input dictionary and returns the new dictionary

In [36]:
def removeDuplicateWords(classes):
  booleanClasses={}
  for className in classes:
    textsList=copy.deepcopy(classes[className])
    newtextsList=[]
    for texts in textsList:
      newtext=""
      allWords=texts.split()
      temp = []
      for word in allWords:
        if word not in temp:
            temp.append(word)
      newtext= ' '.join(temp)
      newtextsList.append(newtext)
    booleanClasses[className]=newtextsList

  return booleanClasses

Removing duplicate words from the training data set and storing in new dictionary

In [37]:
booleanClasses=removeDuplicateWords(classes)

Will take approximately 5-6 minutes to execute

In [38]:
vocabulary, classPriorBNB, wordProbabilitiesByClassBNB=NaiveBaiseTraining(booleanClasses)
print(len(wordProbabilitiesByClassBNB['Real']))
print(wordProbabilitiesByClassBNB['Fake']['جس'])
print(wordProbabilitiesByClassBNB['Real']['جس'])

In class  Real
In class  Fake
In class  Real
Vocabulary size:  13750
In class  Fake
Vocabulary size:  13750
13750
0.0032582887051768664
0.002679695218503431


# Testing

Getting the list of correct labels for the documents

In [46]:
correctLabels=[]
for name in testClasses:
  for text in testClasses[name]:
    correctLabels.append(name)


## Testing the Normal Naive Bayes Classifier

Getting list of predicted labels

In [53]:
NBPredictedLabels=[]
for name in testClasses:
  for text in testClasses[name]:
    predicted=naiveBayesTest(testClasses, vocabulary, classPrior, wordProbabilitiesByClass, text)
    NBPredictedLabels.append(predicted)
print(len(NBPredictedLabels))

262


## Testing the Classifier with Stop Words Removed from Test and Training Sets

Getting list of predicted labels

In [101]:
SWPredictedLabels=[]
for name in stopWordsRemovedTest:
  for text in stopWordsRemovedTest[name]:
    predicted=naiveBayesTest(stopWordsRemovedTest, vocabularySW, classPriorSW, wordProbabilitiesByClassSW, text)
    SWPredictedLabels.append(predicted)
print(len(SWPredictedLabels))

262


## Testing the Classifier with Negations having been Pre Processed (Stop Words Present)

In [91]:
NGPredictedLabels=[]
for name in negationTestingClasses:
  for text in negationTestingClasses[name]:
    predicted=naiveBayesTest(negationTestingClasses, vocabularyNG, classPriorNG, wordProbabilitiesByClassNG, text)
    NGPredictedLabels.append(predicted)
print(len(NGPredictedLabels))

262


## Testing the Classifier with Negations having been Pre Processed and Stop Words Removed

In [92]:
SWNGPredictedLabels=[]
for name in stopWordsNGRemovedTest:
  for text in stopWordsNGRemovedTest[name]:
    predicted=naiveBayesTest(stopWordsNGRemovedTest, vocabularySWNG, classPriorSWNG, wordProbabilitiesByClassSWNG, text)
    SWNGPredictedLabels.append(predicted)
print(len(SWNGPredictedLabels))

262


## Testing the Boolean Naive Bayes Algorithm with Duplicate Words Removed from Training Documents

Getting list of predicted labels

In [52]:
BNBPredictedLabels=[]
for name in testClasses:
  for text in testClasses[name]:
    predicted=naiveBayesTest(testClasses, vocabulary, classPriorBNB, wordProbabilitiesByClassBNB, text)
    BNBPredictedLabels.append(predicted)
print(len(BNBPredictedLabels))

262


# Analysis

## Comparing Accuracy of the Five Classifiers

In [102]:
algorithms=['Normal Naive Bayes', 'StopWords Removed', 'Boolean Naive Bayes', 'PreProcessed Negations', 'PreProcessed Negations and StopWords Removed']

In [107]:
NBAccuracy=sk.accuracy_score(correctLabels, NBPredictedLabels)
SWAccuracy=sk.accuracy_score(correctLabels, SWPredictedLabels)
BNBAccuracy=sk.accuracy_score(correctLabels, BNBPredictedLabels)
NGAccuracy=sk.accuracy_score(correctLabels, NGPredictedLabels)
SWNGAccuracy=sk.accuracy_score(correctLabels, SWNGPredictedLabels)

print("The accuracy for the normal Naive Bayes Algorithm is ", NBAccuracy)
print("The accuracy for when StopWords are removed is ", SWAccuracy)
print("The accuracy for Boolean Naive Bayes is ", BNBAccuracy)
print("The accuracy for when Negations are preprocessed is ", NGAccuracy)
print("The accuracy for when Negations are preprocessed and StopWords removed is ", SWNGAccuracy)
print("")
print("Thus the best accuracy is of ", algorithms[np.argmax([NBAccuracy, SWAccuracy, BNBAccuracy, NGAccuracy, SWNGAccuracy])])

The accuracy for the normal Naive Bayes Algorithm is  0.7022900763358778
The accuracy for when StopWords are removed is  0.6946564885496184
The accuracy for Boolean Naive Bayes is  0.7175572519083969
The accuracy for when Negations are preprocessed is  0.6984732824427481
The accuracy for when Negations are preprocessed and StopWords removed is  0.6870229007633588

Thus the best accuracy is of  Boolean Naive Bayes


## Comparing the Precision of the Five Classifiers

In [108]:
NBPrecisionR=sk.precision_score(correctLabels, NBPredictedLabels, average='macro')
SWPrecisionR=sk.precision_score(correctLabels, SWPredictedLabels, average='macro')
BNBPrecisionR=sk.precision_score(correctLabels, BNBPredictedLabels, average='macro')
NGPrecision=sk.precision_score(correctLabels, NGPredictedLabels, average='macro')
SWNGPrecision=sk.precision_score(correctLabels, SWNGPredictedLabels, average='macro')

print("The precision for the normal Naive Bayes Algorithm is ", NBPrecisionR)
print("The precision for when StopWords are removed is ", SWPrecisionR)
print("The precision for Boolean Naive Bayes with is ", BNBPrecisionR)
print("The precision for when Negations are preprocessed is ", NGPrecision)
print("The precision for when Negations are preprocessed and StopWords removed is ", SWNGPrecision)
print("")
print("Thus the best precision is of ", algorithms[np.argmax([NBPrecisionR, SWPrecisionR, BNBPrecisionR, NGPrecision, SWNGPrecision])])


The precision for the normal Naive Bayes Algorithm is  0.6956548198636806
The precision for when StopWords are removed is  0.6878588516746411
The precision for Boolean Naive Bayes with is  0.7129694835680751
The precision for when Negations are preprocessed is  0.6916189327705757
The precision for when Negations are preprocessed and StopWords removed is  0.6800239234449761

Thus the best precision is of  Boolean Naive Bayes


## Computing Recall Score of the Five Classifiers

In [109]:
NBf1r=sk.recall_score(correctLabels, NBPredictedLabels, average='macro')
SWf1r=sk.recall_score(correctLabels, SWPredictedLabels, average='macro')
BNBf1r=sk.recall_score(correctLabels, BNBPredictedLabels, average='macro')
NGf1r=sk.recall_score(correctLabels, NGPredictedLabels, average='macro')
SWNGf1r=sk.recall_score(correctLabels, SWNGPredictedLabels, average='macro')

print("The recall score for the normal Naive Bayes Algorithm is ", NBf1r)
print("The recall score for when StopWords are removed is ", SWf1r)
print("The recall score for Boolean Naive Bayes is ", BNBf1r)
print("The recall score for when Negations are preprocessed is ", NGf1r)
print("The recall score for when Negations are preprocessed and StopWords removed is ", SWNGf1r)
print("")
print("Thus the best recall score is of ", algorithms[np.argmax([NBf1r, SWf1r, BNBf1r, NGf1r, SWNGf1r])])


The recall score for the normal Naive Bayes Algorithm is  0.6913690476190477
The recall score for when StopWords are removed is  0.6869642857142857
The recall score for Boolean Naive Bayes is  0.7160119047619048
The recall score for when Negations are preprocessed is  0.6891666666666667
The recall score for when Negations are preprocessed and StopWords removed is  0.6791666666666667

Thus the best recall score is of  Boolean Naive Bayes


## Comparing the F1 Scores of the Three Algorithms

In [110]:
NBf1r=sk.f1_score(correctLabels, NBPredictedLabels, average='macro')
SWf1r=sk.f1_score(correctLabels, SWPredictedLabels, average='macro')
BNBf1r=sk.f1_score(correctLabels, BNBPredictedLabels, average='macro')
NGf1r=sk.f1_score(correctLabels, NGPredictedLabels, average='macro')
SWNGf1r=sk.f1_score(correctLabels, SWNGPredictedLabels, average='macro')

print("The F1 score for the normal Naive Bayes Algorithm is ", NBf1r)
print("The F1 score for when StopWords are removed is ", SWf1r)
print("The recall score for Boolean Naive Bayes is ", BNBf1r)
print("The F1 score for when Negations are preprocessed is ", NGf1r)
print("The F1 score for when Negations are preprocessed and StopWords removed is ", SWNGf1r)
print("")
print("Thus the best F1 score is of ", algorithms[np.argmax([NBf1r, SWf1r, BNBf1r, NGf1r, SWNGf1r])])


The F1 score for the normal Naive Bayes Algorithm is  0.6928210678210678
The F1 score for when StopWords are removed is  0.6873694886939921
The recall score for Boolean Naive Bayes is  0.7138049126121871
The F1 score for when Negations are preprocessed is  0.6901265064750355
The F1 score for when Negations are preprocessed and StopWords removed is  0.6795537259113418

Thus the best F1 score is of  Boolean Naive Bayes


According to the overall accuracy scores above as well as Precision, Recall and F1, the best performance is by the Boolean Naive Bayes although there is only a fractional improvement as compared to the normal Naive Bayes algorithm.

When only negations are pre-processed using the above mentioned approach, the performance decreases fractionally from the normal Naive Bayes Classifier and thus this performs the third best.

When Stop Words are removed from the training and testing sets, the performance actually decreases as compared to when they are kept both in Normal and PreProcessed Negations.

The least best performance is when both negations are preprocessed and stop words removed as well.

In order of performance the classifiers can be sorted from best to worst as:

Boolean Naive Bayes

Normal Naive Bayes

Pre-Processed Negations

StopWords Removed

Pre-Processed Negations and Stop Words Removed